In [5]:
import pandas as pd
import requests
import os
import time

matches = pd.read_csv('WCMatches.csv')
matches.columns
matches[["winnerid","loserid"]]




,winnerid,loserid
0,7d05pfwn,jfvflnna
1,o8q1tnkp,o8q1tnkp
2,akq5t76h,akq5t76h
3,ng9mpb4s,ng9mpb4s
4,ng9mpb4s,ng9mpb4s
...,...,...
2783,dq44xncy,222vn7x7
2784,g7umra42,g7umra42
2785,g7umra42,222vn7x7
2786,g7umra42,dq44xncy


In [11]:
# Ensure unique_ids is unique
unique_ids = pd.concat([matches['winnerid'], matches['loserid']]).unique()
print(unique_ids.size)
player_info = pd.DataFrame()

1210


The following is only for one time execution to get the first info of all players from the inofficial WC

In [12]:
import os
# Load existing player_info if it exists, otherwise create empty DataFrame
player_info_path = "EloPlayerinfo.csv"
if os.path.exists(player_info_path):
    player_info = pd.read_csv(player_info_path, low_memory=False)
    processed_ids = set(player_info['player_id'].unique())
else:
    player_info = pd.DataFrame()
    processed_ids = set()

# Only process player_ids that are not already in player_info
to_process = [pid for pid in unique_ids if pid not in processed_ids]

for idx, player_id in enumerate(to_process, start=len(processed_ids)):
    url_events = f"https://mtgeloproject.net/api/players/{player_id}/events"
    url_matches = f"https://mtgeloproject.net/api/players/{player_id}/matches"
    response_events = requests.get(url_events)
    response_matches = requests.get(url_matches)
    
    # Convert responses to JSON
    events_data = response_events.json()
    matches_data = response_matches.json()

    #flatten and rename events
    events_data_df = pd.DataFrame(events_data['data'])
    events_data_df = events_data_df.rename(columns={"code":"tournament_code"})

    # Flatten matches_data so each match is a row, including tournament code
    all_matches = []
    for tournament_code, matches_list in matches_data.items():
        for match in matches_list:
            match_copy = match.copy()
            match_copy['tournament_code'] = tournament_code
            all_matches.append(match_copy)
    matches_data_flat_df = pd.DataFrame(all_matches)

    # Expand own_elo and opp_data columns into separate columns and drop duplicate column
    matches_data_flat_df = matches_data_flat_df.join(pd.json_normalize(matches_data_flat_df.pop('own_elo')).add_prefix('own_elo_'))
    matches_data_flat_df = matches_data_flat_df.join(pd.json_normalize(matches_data_flat_df.pop('opp_data')).add_prefix('opp_'))
    matches_data_flat_df = matches_data_flat_df.drop(columns = "format")

    # Merge matches and events on event code
    merged_df = pd.merge(matches_data_flat_df, events_data_df, on="tournament_code", how="left", copy=False)
    #add player id
    merged_df.insert(loc=0, column= "player_id", value=player_id)

    # Append merged_df to player_info
    player_info = pd.concat([player_info, merged_df], ignore_index=True)

    print(f"Index: {idx}, Player ID: {player_id}")
    time.sleep(0.5) #wait 0.5 second to not pummel the server

# Save player_info after all data is collected
player_info.to_csv("EloPlayerinfo.csv", index=False)
print("List done")

List done


In [74]:
#find if new id's that have not been in the list before have come up
player_info_unique_opps = player_info[["opp_id", "opp_opp"]].drop_duplicates()
matches_unique_players = matches[["winnerid","Target"]].drop_duplicates()
matches_unique_opps = matches[["loserid","Source"]].drop_duplicates()

# Combine all into a single DataFrame with columns: id, name
all_pairs = pd.concat([
    player_info_unique_opps.rename(columns={"opp_id": "id", "opp_opp": "name"}),
    matches_unique_players.rename(columns={"winnerid": "id", "Target": "name"}),
   matches_unique_opps.rename(columns={"loserid": "id", "Source": "name"})
], ignore_index=True)

# Drop duplicates and rows with missing names or ids
all_pairs = all_pairs.dropna(subset=["id", "name"]).drop_duplicates()

# Save the result
all_pairs.to_csv("UniqueIdNamePairs.csv", index=False)


unique_ids_update = pd.DataFrame(all_pairs["id"].unique())
print(unique_ids_update.size)
unique_ids_update.to_csv("UniqueIds.csv", index=False)



90464


This is to make it repeatable

In [2]:

def get_Player_Events_Matches_from_URL(player_id):
    url_events = f"https://mtgeloproject.net/api/players/{player_id}/events"
    url_matches = f"https://mtgeloproject.net/api/players/{player_id}/matches"
    response_events = requests.get(url_events)
    response_matches = requests.get(url_matches)
    
    # Convert responses to JSON
    events_data = response_events.json()
    matches_data = response_matches.json()

    #flatten and rename events
    events_data_df = pd.DataFrame(events_data['data'])
    events_data_df = events_data_df.rename(columns={"code":"tournament_code"})

    # Flatten matches_data so each match is a row, including tournament code
    all_matches = []
    for tournament_code, matches_list in matches_data.items():
        for match in matches_list:
            match_copy = match.copy()
            match_copy['tournament_code'] = tournament_code
            all_matches.append(match_copy)
    matches_data_flat_df = pd.DataFrame(all_matches)

    # Expand own_elo and opp_data columns into separate columns and drop duplicate column
    matches_data_flat_df = matches_data_flat_df.join(pd.json_normalize(matches_data_flat_df.pop('own_elo')).add_prefix('own_elo_'))
    matches_data_flat_df = matches_data_flat_df.join(pd.json_normalize(matches_data_flat_df.pop('opp_data')).add_prefix('opp_'))
    matches_data_flat_df = matches_data_flat_df.drop(columns = "format")

    # Merge matches and events on event code
    merged_df = pd.merge(matches_data_flat_df, events_data_df, on="tournament_code", how="left", copy=False)
    #print(f"Player ID: {player_id} Events and Matches collected")
    return merged_df

In [3]:

import requests
from bs4 import BeautifulSoup
import json

def get_player_data_from_url(id):
    """
    Fetches HTML from a URL, finds the correct astro-island element,
    and extracts player data from its props attribute.
    """
    url = f"https://mtgeloproject.net/profile/{id}"
    try:
        # Fetch the HTML content from the URL
        response = requests.get(url)
        response.raise_for_status() # Raise an exception for bad status codes

        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')

        # Find the astro-island element that corresponds to the Profile component.
        # We'll use the 'component-url' attribute to find it.
        # You might need to adjust the component_url value if it changes.
        # Alternatively, you could use 'component-export': 'default'
        # or look for a unique data attribute if available.
        component_url_to_find = '/_astro/Profile.FIYvQsk3.js'
        astro_island_element = soup.find('astro-island', {'component-url': component_url_to_find})


        if astro_island_element:
            props_attribute = astro_island_element.get('props')
            if props_attribute:
                try:
                    # The props attribute value is a JSON string
                    # In the provided data, the structure seems to be a list where the second element
                    # contains the 'info' object.
                    props_data = json.loads(props_attribute)

                    # Accessing the info object. Based on your provided data structure,
                    # the 'info' object is the second element in the main list value
                    # associated with the key "info".
                    info_data = props_data.get('info', [None, {}])[1]

                    player_id = info_data.get('id', [None, None])[1]
                    first_name = info_data.get('first_name', [None, None])[1]
                    last_name = info_data.get('last_name', [None, None])[1]

                    #print(f"ID: {player_id}")
                    #print(f"First Name: {first_name}")
                    #print(f"Last Name: {last_name}")
                    info = [player_id,last_name,first_name]
                    return info
                    

                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON from props attribute: {e}")
                except Exception as e:
                    print(f"An error occurred while processing data: {e}")
            else:
                print("Error: astro-island element found, but no 'props' attribute.")
        else:
            print(f"Error: Could not find an astro-island element with component-url '{component_url_to_find}'.")
            print("Consider adjusting the selector or trying an alternative method (commented out in the script).")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching the URL: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


In [25]:
test_ids = ["l8ugp31t","5ysod3tm"]
#,"h0x63109","l8re36n5"
player_data=[]
for id in test_ids:
    matches = get_Player_Events_Matches_from_URL(id)
    name = get_player_data_from_url(id)
    print(name)
    print(matches)
    L =[name + list(row) for row in matches.values]
    
    player_data.extend(L)

player_data_df = pd.DataFrame(player_data)
player_data_df.to_csv("complete_data_df.csv", index= False)
#print(player_data)

Player ID: l8ugp31t Events and Matches collected
['l8ugp31t', 'Croft', 'Michael']
    match_id round  table    result tournament_code  own_elo_start  \
0       5806     1     99   Won 0-0        ptla9899        1500.00   
1       5901     2     29   Won 2-1        ptla9899        1518.00   
2       6098     3     62  Lost 0-2        ptla9899        1536.66   
3       6228     4     28  Lost 1-2        ptla9899        1523.10   
4       6444     5     83  Lost 0-2        ptla9899        1504.43   
..       ...   ...    ...       ...             ...            ...   
61     86212     6      1   Won 2-1         natca01        1547.36   
62     86280     7     21  Lost 0-2         natca01        1563.93   
63     86337     8     32   Won 2-1         natca01        1548.84   
64     86369     9     21   Won 2-1         natca01        1565.90   
65     86403    10     19  Lost 1-2         natca01        1582.71   

    own_elo_end    opp_id           opp_opp  opp_start cut_before        date

In [ ]:
import os
import pandas as pd
import time
import logging
import subprocess

# Set up logging
logging.basicConfig(filename='scrape_log2.txt', level=logging.INFO, format='%(asctime)s - %(message)s')
logging.info(f"Start log")

header_written = False


def load_existing_data(player_list_path, unique_ids_update_path):
    """Load existing data for players and return processed IDs"""
    unique_ids_update = pd.read_csv(unique_ids_update_path, header=None)
    if os.path.exists(player_list_path):
        unique_player_list = pd.read_csv(player_list_path, low_memory=False)
        processed_ids = set(unique_player_list['0'].unique())
        global header_written
        header_written = True
    else:
        unique_player_list = pd.DataFrame()
        processed_ids = set()
    
    return unique_ids_update, processed_ids

def run_git_lfs_prune():
    """Run git lfs prune to remove old LFS files"""
    try:
        logging.info("Running git lfs prune to clean up LFS storage...")
        # Run the git lfs prune command with shell=True for Windows compatibility
        subprocess.run("git lfs prune", check=True, shell=True)
        logging.info("git lfs prune completed successfully.")
    except subprocess.CalledProcessError as e:
        logging.error(f"Failed to run git lfs prune: {e}")
        print(f"Error running git lfs prune: {e}")

def save_to_csv(player_list_path, player_data_list, header=False):
    """Save player data to CSV"""
    if player_data_list:
        try:
            player_df = pd.DataFrame(player_data_list)
            player_df.to_csv(player_list_path, mode='a', header=header, index=False)
            global header_written
            header_written = True
        except OSError as e:
            # If the error is due to disk space (Errno 28: No space left on device)
            if e.errno == 28:
                logging.error("Disk space full. Running git lfs prune...")
                print("Disk space full. Running git lfs prune...")
                run_git_lfs_prune()
                # Retry saving after running git lfs prune
                try:
                    player_df.to_csv(player_list_path, mode='a', header=header, index=False)
                    logging.info("Data saved successfully after running git lfs prune.")
                except Exception as e:
                    logging.error(f"Failed to save data after pruning: {e}")
                    print(f"Failed to save data after pruning: {e}")
            else:
                logging.error(f"Failed to save data: {e}")
                print(f"Failed to save data: {e}")


def process_players_in_batches(unique_ids_update, processed_ids, player_list_path, batch_size=50):
    """Process players in batches and save data periodically"""
    player_data = []
    to_process = [pid for pid in unique_ids_update[0] if pid not in processed_ids]
    total_batches = len(to_process) // batch_size + (1 if len(to_process) % batch_size != 0 else 0)

    try:
        for batch_idx in range(total_batches):
            start_idx = batch_idx * batch_size
            end_idx = min((batch_idx + 1) * batch_size, len(to_process))
            batch_ids = to_process[start_idx:end_idx]

            process_batch(batch_ids, player_data)

            # Save batch data
            save_to_csv(player_list_path, player_data, header=not header_written)
            
            print(f"Saved batch {batch_idx + 1}/{total_batches} after processing Player IDs {batch_ids}")
            logging.info(f"Saved batch {batch_idx + 1}/{total_batches} after processing Player IDs {batch_ids}")
            player_data = []  # Clear data for next batch

    except KeyboardInterrupt:
        # Handle graceful stop when interrupted (Ctrl+C)
        logging.info("Process interrupted. Saving current data before exit...")
        if player_data:
            save_to_csv(player_list_path, player_data, header=not header_written)
            logging.info(f"Saved remaining {len(player_data)} players before stopping.")
        print("Process interrupted. Current data saved.")

def process_batch(batch_ids, player_data):
    """Process each batch of player IDs"""
    for idx, id in enumerate(batch_ids):
        try:
            #fetch data
            matches = get_Player_Events_Matches_from_URL(id)
            name = get_player_data_from_url(id)
            #merge name and match data
            l = [name + list(row) for row in matches.values]
            player_data.extend(l)
            print(f"Index {idx}: Player {id} processed")
            time.sleep(0.1)  # Throttle requests

        except Exception as e:
            logging.error(f"Error processing Player ID {id}: {e}")
            print(f"Error processing Player ID {id}: {e}")
            continue  # Skip to the next player ID on error

        logging.info(f"Successfully processed Player ID {id}")

# Main processing flow
player_list_path = "UniquePlayerList2.csv"
unique_ids_update_path = "UniqueIds.csv"

# Load existing data
unique_ids_update, processed_ids = load_existing_data(player_list_path, unique_ids_update_path)

# Process players in batches and save data periodically
process_players_in_batches(unique_ids_update, processed_ids, player_list_path, batch_size=50)

print("Processing complete.")


Error processing Player ID 0: 'NoneType' object is not iterable
Index 1: Player iv0tyslq processed
Index 2: Player 2oaejlj1 processed
Index 3: Player 22e5wiuz processed
Index 4: Player zxpmnkkj processed
Index 5: Player x3lhd0tb processed
Index 6: Player ox64yvhn processed
Index 7: Player mrpoxq2s processed
Index 8: Player ltfpxf4g processed
Index 9: Player aibewjrn processed
Index 10: Player n0u2aqdk processed
Index 11: Player 9o0b3tl5 processed
Index 12: Player ersy1w0g processed
Index 13: Player atwzhs0r processed
Index 14: Player bzfdxczf processed
Index 15: Player u8bw4hah processed
Index 16: Player feywb8v6 processed
Index 17: Player nz9e9m6r processed
Index 18: Player jwstjxa0 processed
Index 19: Player 1bfwvk1k processed
Index 20: Player ju03m3cb processed
Index 21: Player 2urls15z processed
Index 22: Player jwtjxjb9 processed
Index 23: Player 8ugfdreq processed
Index 24: Player zzlqwnsk processed
Index 25: Player tld8nnq8 processed
Index 26: Player 43wqzagi processed
Index 27:

In [118]:
print(player_list)
import csv
with open('PlayerList.csv', mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write the list as a row in the CSV file
    writer.writerow(player_list)
#player_list_df = pd.DataFrame(player_list, columns=["ID","LastName","FirstName"])
#player_list_df.to_csv("UniquePlayerList.csv",index=False)

[['f917lein', 'Swarowski', 'Robert'], ['yqvwjvgy', 'Pisicchio', 'Gabriele'], ['jkdsvj4j', 'Ishida', 'Itaru'], ['fodt8dk9', 'Grubaugh', 'Ed'], ['k9jc66ma', 'Sikorsky', 'Claudio'], ['tga82sa2', 'Franzen', 'Johan'], ['6hrzkc4m', 'Turian', 'Michael'], ['x1lebg8w', 'Kibler', 'Brian'], ['l98twjlh', 'Nishioka', 'Andrew'], ['imygnksk', 'Phillipps', 'Eric'], ['i2kzizct', 'Matarese', 'Chris'], ['sannwwjw', 'Schraut', 'Andreas'], ['h6ke3hcc', 'Wolf', 'Andrew'], ['r99or99j', 'Ladra', 'Lukas'], ['lewcamij', 'Ogilbee', 'Brian'], ['d7k7077v', 'Glodt', 'Romain'], ['me4nlw5o', 'Guevin', 'Thomas'], ['pzh1wbfb', 'Moss-Allison', 'Nicolas'], ['6mia5w8m', 'Konstanczer', 'Andre'], ['897497lk', 'Chinnock', 'John'], ['gpnoyxve', 'Eskeland', 'Sigurd'], ['lvzgd7gn', 'Eikefet', 'Kim'], ['z2315xnw', 'Slemr', 'Jakub'], ['tzu75jdp', 'Selden', 'Brian'], ['pzh3fcim', 'Levy', 'Raphael'], ['akq5t76h', 'Rubin', 'Benjamin'], ['3eudavoh', 'Johns', 'Scott'], ['nwnjg4a4', 'Buehler', 'Randy'], ['er8rx6et', 'Rodrigues Leone', 